In [1]:
import cv2
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import os

In [2]:
def load_data():
    np.random.seed(42)  # For reproducibility
    data_size = 2000
    X = np.random.rand(data_size, 64 * 64)  # Simulated 64x64 grayscale image features
    y_gender = np.random.choice(['Male', 'Female'], size=data_size)
    y_age = np.random.choice(['Child', 'Adult', 'Senior'], size=data_size)

In [3]:
def load_data():
    np.random.seed(42)  # For reproducibility
    data_size = 2000
    X = np.random.rand(data_size, 64 * 64)  # Simulated 64x64 grayscale image features
    y_gender = np.random.choice(['Male', 'Female'], size=data_size)
    y_age = np.random.choice(['Child', 'Adult', 'Senior'], size=data_size)

    # Encode labels
    gender_encoder = LabelEncoder()
    age_encoder = LabelEncoder()
    y_gender = gender_encoder.fit_transform(y_gender)
    y_age = age_encoder.fit_transform(y_age)

    X_train, X_test, y_gender_train, y_gender_test, y_age_train, y_age_test = train_test_split(
        X, y_gender, y_age, test_size=0.7, random_state=42
    )
    return X_train, X_test, y_gender_train, y_gender_test, y_age_train, y_age_test, gender_encoder, age_encoder

In [4]:
class GenderAgeModel:
    def __init__(self):
        self.gender_model = DecisionTreeClassifier(max_depth=10, random_state=42)
        self.age_model = DecisionTreeClassifier(max_depth=10, random_state=42)

    def train(self, X_train, y_gender_train, y_age_train):
        self.gender_model.fit(X_train, y_gender_train)
        self.age_model.fit(X_train, y_age_train)

    def predict(self, X):
        gender_pred = self.gender_model.predict(X)
        age_pred = self.age_model.predict(X)
        return gender_pred, age_pred

    def accuracy(self, X_test, y_gender_test, y_age_test):
        gender_acc = accuracy_score(y_gender_test, self.gender_model.predict(X_test))
        age_acc = accuracy_score(y_age_test, self.age_model.predict(X_test))
        return gender_acc, age_acc


In [5]:
X_train, X_test, y_gender_train, y_gender_test, y_age_train, y_age_test, gender_encoder, age_encoder = load_data()
model = GenderAgeModel()
model.train(X_train, y_gender_train, y_age_train)

In [6]:
gender_acc, age_acc = model.accuracy(X_test, y_gender_test, y_age_test)
print(f"Gender Model Accuracy: {gender_acc:.2f}")
print(f"Age Model Accuracy: {age_acc:.2f}")

Gender Model Accuracy: 0.50
Age Model Accuracy: 0.33


In [ ]:
def live_detection(model, gender_encoder, age_encoder):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            resized = cv2.resize(face, (64, 64)).flatten().reshape(1, -1)

            gender_pred, age_pred = model.predict(resized)
            gender_label = gender_encoder.inverse_transform([gender_pred[0]])[0]
            age_label = age_encoder.inverse_transform([age_pred[0]])[0]

            label = f"Gender: {gender_label}, Age: {age_label}"
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        cv2.imshow("Gender and Age Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

live_detection(model, gender_encoder, age_encoder)